### <center>RECOMMENDER SYSTEM</center>

### Data Editing

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

from google.colab import drive
drive.mount('/content/drive')

movies_data = pd.read_csv('/content/drive/MyDrive/Portfolio/1. Capstone Projects/Recommender System/movies.csv')

movies_data['production_company'] = movies_data['production_companies'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])
movies_data['production_country'] = movies_data['production_countries'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])
movies_data['crews'] = movies_data['crew'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x,list) else [])
movies_data['production_company'] = movies_data['production_company'].apply(lambda x: ' '.join(map(str,x)))
movies_data['production_country'] = movies_data['production_country'].apply(lambda x: ' '.join(map(str,x)))
movies_data['crews'] = movies_data['crews'].apply(lambda x: ' '.join(map(str,x)))


Mounted at /content/drive


## Method1:

In [2]:
selected_features = ['genres','keywords','tagline','cast','director','production_company','production_country','crews']

for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']+' '+movies_data['production_company']+' '+movies_data['production_country']+' '+movies_data['crews']


vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

similarity = cosine_similarity(feature_vectors)

movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)  

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
    index = movie[0]
    title_from_index = movies_data[movies_data.index==index]['title'].values[0]
    if (i<31):
        print(i, '.',title_from_index)
        i+=1



 Enter your favourite movie name : Forrest Gump
Movies suggested for you : 

1 . Forrest Gump
2 . Contact
3 . What Lies Beneath
4 . The Polar Express
5 . Beowulf
6 . Cast Away
7 . Back to the Future Part III
8 . Back to the Future Part II
9 . Flight
10 . Death Becomes Her
11 . The Walk
12 . 15 Minutes
13 . Road to Perdition
14 . The Faculty
15 . A Christmas Carol
16 . Toy Story
17 . War of the Worlds
18 . Back to the Future
19 . House on Haunted Hill
20 . Jay and Silent Bob Strike Back
21 . The Alamo
22 . Terminator 2: Judgment Day
23 . Big Fish
24 . Independence Day
25 . Meet Joe Black
26 . A.I. Artificial Intelligence
27 . Eraser
28 . The Wolf of Wall Street
29 . Armageddon
30 . The Sixth Sense


-------------------------------------------------------------------------------

## Method2:

In [3]:
movies_data['tagline'] = movies_data['tagline'].fillna('')
movies_data['description'] = movies_data['overview'] + movies_data['tagline']
movies_data['description'] = movies_data['description'].fillna('')

In [4]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies_data['description'])

In [5]:
tfidf_matrix.shape

(4803, 149317)

In [6]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
cosine_sim[0]

array([1.        , 0.00449945, 0.        , ..., 0.00272894, 0.        ,
       0.        ])

In [8]:
titles = movies_data['title']
indices = pd.Series(movies_data.index, index=movies_data['title'])

In [9]:
indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [10]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [13]:
get_recommendations('Forrest Gump').head(30)

2686                    An American Haunting
124                                   Frozen
1714                             Serendipity
1495                              Phenomenon
3866                             City of God
4603                            My Dog Tulip
4228             The Ballad of Jack and Rose
1559                            The Notebook
1259             Memoirs of an Invisible Man
620                              The Insider
4287                                 Niagara
2260                         All Good Things
2971                               Housefull
776                            Winter's Tale
2797                  Lars and the Real Girl
4021                              Brick Lane
719                               This Is It
4791                             Tin Can Man
2760                                    Room
1957                      Dangerous Liaisons
2757                   Letters from Iwo Jima
835                    Mr. Popper's Penguins
2904      

In [14]:
get_recommendations('The Dark Knight').head(30)

3                         The Dark Knight Rises
299                              Batman Forever
428                              Batman Returns
3854    Batman: The Dark Knight Returns, Part 2
1359                                     Batman
2507                                  Slow Burn
1181                                        JFK
119                               Batman Begins
879                         Law Abiding Citizen
205          Sherlock Holmes: A Game of Shadows
9            Batman v Superman: Dawn of Justice
2193                       Secret in Their Eyes
210                              Batman & Robin
2020                                 The Rookie
58                                  Rush Hour 3
2207                                  12 Rounds
4135            Gangster's Paradise: Jerusalema
2692                                    Thinner
3725                                The Sweeney
506                             Despicable Me 2
2464                     The Master of D

# Conclusion 1:

#### So far, method 1 is recommending expected movies.